In [30]:
# Importar bibliotecas necesarias
import pandas as pd
import mysql.connector



ModuleNotFoundError: No module named 'mysql'

In [ ]:

# Leer el archivo CSV
df = pd.read_csv('../data/requests.csv')


# Mostrar las primeras filas para verificar los datos
df.head()

In [ ]:
# Verificar si hay valores nulos
print("Valores nulos por columna:")
print(df.isnull().sum())

# Eliminar duplicados si existen
df = df.drop_duplicates()

# Verificar tipos de datos
print("Tipos de datos:")
print(df.dtypes)

# Filtrar datos con fechas inválidas
df = df[df['Fecha'].notnull()]

# Volver a mostrar las primeras filas tras la limpieza
df.head()


# Transformaciones con Pandas

In [ ]:

# Convertir la fecha de la solicitud a formato datetime
df['Fecha'] = pd.to_datetime(df['F.solicitud'], errors='coerce')

# Crear columnas nuevas de mes y año
df['mes'] = df['Fecha'].dt.month
df['año'] = df['Fecha'].dt.year


Cálculo de indicadores

In [ ]:
# KPI Ejemplo: ¿Resoluciones rápidas?
df['resolucion_rapida'] = df['Estado'].apply(lambda x: 1 if x == 'resuelto en menos de 5 días' else 0)

# Ver el dataframe con las nuevas columnas y KPIs
df.head()

In [ ]:
# Conexión a MySQL
DB_CONFIG = {
    'host': 'localhost',
    'user': 'user',
    'password': 'password',
    'database': 'data_db'
    }

 KPI 1: Tiempo promedio de resolución

Supongamos que tienes una columna llamada F.resolucion que indica la fecha en que la solicitud fue resuelta.

In [ ]:
# Convertir la fecha de resolución a formato datetime
df['Fecha_resolucion'] = pd.to_datetime(df['F.resolucion'], errors='coerce')

# Calcular el tiempo de resolución en días
df['tiempo_resolucion'] = (df['Fecha_resolucion'] - df['Fecha']).dt.days

# Calcular el promedio de tiempo de resolución
tiempo_promedio_resolucion = df['tiempo_resolucion'].mean()
print(f"Tiempo promedio de resolución: {tiempo_promedio_resolucion} días")


KPI 2: Solicitudes resueltas por mes/año

In [ ]:
# Filtrar las solicitudes que han sido resueltas
df_resueltas = df[df['Estado'].str.contains('resuelto', case=False, na=False)]

# Agrupar por mes y año para obtener el total de solicitudes resueltas
solicitudes_resueltas_mes_año = df_resueltas.groupby(['mes', 'año']).size().reset_index(name='total_resueltas')

# Ver el resultado
solicitudes_resueltas_mes_año.head()


KPI 3: Tasa de resolución rápida

In [ ]:
# Calcular la tasa de resoluciones rápidas
tasa_resolucion_rapida = df['resolucion_rapida'].mean() * 100
print(f"Tasa de resolución rápida: {tasa_resolucion_rapida:.2f}%")


In [ ]:
def load_data_to_mysql(df):
    connection = mysql.connector.connect(
        host=DB_CONFIG['host'],
        user=DB_CONFIG['user'],
        password=DB_CONFIG['password'],
        database=DB_CONFIG['database']
    )
    cursor = connection.cursor()

    # Insertar los datos procesados en la tabla MySQL
    for _, row in df.iterrows():
        cursor.execute("""
            INSERT INTO solicitudes_procesadas 
            (tipo, estado, fecha_solicitud, origen, unidad, provincia, referencia, mes, año, resolucion_rapida, tiempo_resolucion) 
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            row['Tipo'], row['Estado'], row['Fecha'], row['Origen'], row['Unidad'], 
            row['Provincia'], row['Referencia'], row['mes'], row['año'], 
            row['resolucion_rapida'], row['tiempo_resolucion']
        ))

    # Confirmar la transacción
    connection.commit()
    cursor.close()
    connection.close()
    print("Datos cargados en MySQL correctamente.")


In [ ]:
def save_kpis_to_mysql(kpi_data):
    connection = mysql.connector.connect(
        host=DB_CONFIG['host'],
        user=DB_CONFIG['user'],
        password=DB_CONFIG['password'],
        database=DB_CONFIG['database']
    )
    cursor = connection.cursor()

    # Crear tabla si no existe
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS kpi_indicadores (
            id INT AUTO_INCREMENT PRIMARY KEY,
            tiempo_promedio_resolucion FLOAT,
            total_resueltas_mes_año INT,
            tasa_resolucion_rapida FLOAT
        );
    """)

    # Insertar los KPIs calculados
    cursor.execute("""
        INSERT INTO kpi_indicadores (tiempo_promedio_resolucion, total_resueltas_mes_año, tasa_resolucion_rapida)
        VALUES (%s, %s, %s);
    """, (
        tiempo_promedio_resolucion, solicitudes_resueltas_mes_año['total_resueltas'].sum(), tasa_resolucion_rapida
    ))

    # Confirmar la transacción
    connection.commit()
    cursor.close()
    connection.close()
    print("KPIs guardados en MySQL correctamente.")

# Guardar los KPIs calculados
save_kpis_to_mysql({
    'tiempo_promedio_resolucion': tiempo_promedio_resolucion,
    'total_resueltas_mes_año': solicitudes_resueltas_mes_año,
    'tasa_resolucion_rapida': tasa_resolucion_rapida
})


In [ ]:

# Ejecutar la función para cargar los datos en MySQL
load_data_to_mysql(df)